<a href="https://colab.research.google.com/github/QuanchaoSong/DemandMarket-Server/blob/master/CLIPImageModel_to_CoreML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install clip-benchmark>=1.4.0 datasets>=2.8.0 open-clip-torch>=2.20.0 timm>=0.9.5 coremltools


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
#We clone MobileCLIP from: https://github.com/apple/ml-mobileclip
!git clone https://github.com/apple/ml-mobileclip.git

#Install MobileCLIP
%cd ml-mobileclip
!pip install -e . -q

In [ ]:
#Download pretrained checkpoints
%mkdir -p checkpoints
!wget wget https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt -P checkpoints

--2024-07-09 13:57:51--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2024-07-09 13:57:51--  https://docs-assets.developer.apple.com/ml-research/datasets/mobileclip/mobileclip_s0.pt
Resolving docs-assets.developer.apple.com (docs-assets.developer.apple.com)... 17.253.7.145, 17.253.7.134, 2620:149:a16:f000::140, ...
Connecting to docs-assets.developer.apple.com (docs-assets.developer.apple.com)|17.253.7.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 215934653 (206M) [application/octet-stream]
Saving to: ‘checkpoints/mobileclip_s0.pt’

mobileclip_s0.pt    100%[===================>] 205.93M   155MB/s    in 1.3s    

2024-07-09 13:57:53 (155 MB/s) - ‘checkpoints/mobileclip_s0.pt’ saved [215934653/215934653]

FINISHED --2024-07-09 13:57:53--
Total wall clock time: 1.7s
Downloaded: 1 files, 206M in 1.3s (155 MB/s)


In [ ]:
#Model inference with reparamerized model
import coremltools
import torch
import mobileclip
from mobileclip.modules.common.mobileone import reparameterize_model
from mobileclip.modules.text.tokenizer import (
    ClipTokenizer,
)
from mobileclip.clip import CLIP
from typing import Dict, Optional, Any
import json
import os

# mobileclip_s0
model_cfg = {
    "embed_dim": 512,
    "image_cfg": {
        "image_size": 256,
        "model_name": "mci0"
    },
    "text_cfg": {
        "context_length": 77,
        "vocab_size": 49408,
        "dim": 512,
        "ffn_multiplier_per_layer": 4.0,
        "n_heads_per_layer": 8,
        "n_transformer_layers": 4,
        "norm_layer": "layer_norm_fp32",
        "causal_masking": False,
        "model_name": "mct"
    }
}

class CLIP_encode_image(CLIP):
    """Class for encoding images using the image encoder from CLIP."""

    def __init__(self, cfg: Dict, output_dict: bool = False, *args, **kwargs) -> None:
        super().__init__(cfg, output_dict, *args, **kwargs)

    def forward(self, image: Optional[torch.Tensor] = None) -> Any:
        image_embeddings = (
            self.encode_image(image, normalize=True) if image is not None else None
        )
        return image_embeddings


model_ie = CLIP_encode_image(cfg=model_cfg)
model_ie.eval()

chkpt = torch.load("checkpoints/mobileclip_s0.pt")
model_ie.load_state_dict(chkpt)

reparameterized_model = reparameterize_model(model_ie)
reparameterized_model.eval()


image = torch.rand(1, 3, 256, 256)
traced_model = torch.jit.trace(reparameterized_model, image)

# Define the input as an image type
input_image = coremltools.ImageType(name="input_image", shape=(1, 3, 256, 256), color_layout=coremltools.colorlayout.RGB, scale=1/255.0, bias=[0, 0, 0])
output_tensor = [coremltools.TensorType(name="output_embeddings")]

ml_model = coremltools.convert(
        model=traced_model,
        outputs=output_tensor,
        inputs=[input_image],
        convert_to="mlprogram",
        minimum_deployment_target=coremltools.target.iOS17,
        compute_units=coremltools.ComputeUnit.ALL,
        debug=True,
    )
ml_model.save("clip_mci_image_s0.mlpackage")

Running MIL default pipeline:  10%|█         | 8/78 [00:00<00:04, 16.43 passes/s]/usr/local/lib/python3.10/dist-packages/coremltools/converters/mil/mil/passes/defs/preprocess.py:266: UserWarning: Output, '1260', of the source model, has been renamed to 'var_1260' in the Core ML model.
  warnings.warn(msg.format(var.name, new_name))
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 35.73 passes/s]


In [ ]:
#Check CoreML Parameters:
spec = ml_model.get_spec()
print("model type: {}".format(spec.WhichOneof('Type')))
print("model description: {}".format(spec.description))
print("model inputs: {}".format(spec.description.input))
print("model outputs: {}".format(spec.description.output))

In [ ]:
#Download mlpackage
import shutil
from google.colab import files
directory_path = 'clip_mci_image_s0.mlpackage'
zip_path = 'clip_mci_image_s0.mlpackage.zip'
shutil.make_archive(directory_path, 'zip', directory_path)
os.rename(f"{directory_path}.zip", zip_path)
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Optional: Visualize the model
!pip install graphviz torchview
from torchview import draw_graph
import graphviz
graphviz.set_jupyter_format('png')
model_graph = draw_graph(reparameterized_model, input_data = image, expand_nested = True, depth = 5)
model_graph.visual_graph

ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'torchview'